In [1]:
#Import your libraries

import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import pylab as plt
import seaborn as sns
import difflib
#%pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [3]:
scrap_cuadros_completo_original = pd.read_csv("/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/data_def/scrap_cuadros_completo.csv")

In [ ]:
dim_original = scrap_cuadros_completo_original.shape # De esta forma guardo las dimensiones del data frame original.
obras_completo_original.shape